In [1]:
from src.resampling import var1_2d_res
from src.generating_data import generate_var1_data
from src.parameters import THETA_HAT_FUNC_DICT
import numpy as np

# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
A = np.array([[0.5, 0.2], [0.1, 0.3]])
Sigma_eps = np.array([[0.1, 0.05], [0.05, 0.1]])
n_obs = 1000
N = 6000

#generate data
X_obs = generate_var1_data(A, Sigma_eps, n_samples=n_obs)
print(f"X_obs.shape: {X_obs.shape}")
A_obs = THETA_HAT_FUNC_DICT['VAR1_A_hat'](X_obs)
# resample data
X_resampled = var1_2d_res(X_obs, N)
A_res = THETA_HAT_FUNC_DICT['VAR1_A_hat'](X_resampled)
# print results
print("A_obs:\n", A_obs)
print("A_res:\n", A_res)


X_obs.shape: (100000, 2)
A_obs:
 [[0.49385399 0.20703789]
 [0.09505055 0.29974599]]
A_res:
 [[0.51333854 0.18640241]
 [0.09876461 0.28721393]]
